In [3]:
import boto3, logging

ddb = boto3.client('dynamodb')

In [4]:
from entities.GatherPosts import GatherPosts

In [6]:
gather_posts = GatherPosts(subreddit="funny", logger=logging.getLogger())

In [8]:
# params = {
#     "TableName": 'DailyUploadsTable-dev',
#     "Item": gather_posts.serialize_to_item(),    
# }

# ddb.put_item(**params)

In [27]:
DAILY_UPLOADS_TABLE_NAME = 'DailyUploadsTable-dev'
params = {
        "TableName": DAILY_UPLOADS_TABLE_NAME,
        "Key": {
            "PK": {"S": '2021-08-17'},
            "SK": {"S": "todays_subreddits_count"},
        },
        "ConditionExpression": "attribute_exists(PK) and attribute_exists(SK)",
        "UpdateExpression": "SET #count = #count + :inc",
        "ExpressionAttributeNames": {"#count": "count"},
        "ExpressionAttributeValues": {":inc": {"N": "1"}},
        "ReturnValues":"NONE"
    }

In [22]:
from helpers import ddb as ddb_helpers
import logging
import pprint

pp = pprint.PrettyPrinter(indent=2, compact=True, width=80)




In [76]:
DAILY_UPLOADS_TABLE_NAME = "DailyUploadsTable-dev"

subreddit_group = ['funny']

transact_items = []
for subreddit in subreddit_group:
    gather_posts = GatherPosts(subreddit,logging.getLogger())
#     gather_posts.date = '2021-08-19'
    transact_item = {
        "Get": {
            "TableName": DAILY_UPLOADS_TABLE_NAME,
            "Key": gather_posts.key()
        }
    }
    
    transact_items.append(transact_item)  
    


kwargs = {'TransactItems' : transact_items}

In [77]:
res = ddb_helpers.transact_get_items(ddb=ddb, logger=logging.getLogger(), **kwargs)

In [78]:
items = [item['Item']  for item in res]

In [79]:
deserialized_items = [
        GatherPosts.deserialize_from_item(item) for item in items
    ]

In [80]:
subreddits = deserialized_items

In [81]:
subreddits[0]

{'posts': [{'duration': 26,
   'upvote_ratio': 0.97,
   'author': 'hey_dont_ban_me_bro',
   'name': 't3_p6u5f4',
   'ups': 5632,
   'title': '"What would you like your obituary to say?" RIP Sean Lock',
   'total_awards_received': 16,
   'url': 'https://v.redd.it/i6ghfjfd35i71'},
  {'duration': 5,
   'upvote_ratio': 0.88,
   'author': 'tacolamp',
   'name': 't3_p6z5nz',
   'ups': 1277,
   'title': 'Parkour!',
   'total_awards_received': 6,
   'url': 'https://v.redd.it/sy8ola7nb6i71'},
  {'duration': 9,
   'upvote_ratio': 0.97,
   'author': 'trytheCOLDchai',
   'name': 't3_p73sa2',
   'ups': 951,
   'title': 'Take a seat, right over here',
   'total_awards_received': 0,
   'url': 'https://v.redd.it/b4j4gz8wj7i71'},
  {'duration': 139,
   'upvote_ratio': 0.97,
   'author': 'NasseHeger',
   'name': 't3_p7bjxc',
   'ups': 2563,
   'title': 'Are any women here today?',
   'total_awards_received': 9,
   'url': 'https://v.redd.it/3uoltpdsaai71'},
  {'duration': 9,
   'upvote_ratio': 0.93,
   '

In [82]:
posts_arr = [
        GatherPosts.deserialize_from_item(item)['posts'] for item in items
    ]

In [84]:
import pandas as pd

In [85]:
df_top = pd.DataFrame()

In [86]:
posts_arr[0][0]

{'duration': 26,
 'upvote_ratio': 0.97,
 'author': 'hey_dont_ban_me_bro',
 'name': 't3_p6u5f4',
 'ups': 5632,
 'title': '"What would you like your obituary to say?" RIP Sean Lock',
 'total_awards_received': 16,
 'url': 'https://v.redd.it/i6ghfjfd35i71'}

In [87]:
for posts in posts_arr:    
    for post in posts:  
        df_top = df_top.append(
            {
                "title": post["title"],
                "upvote_ratio": post["upvote_ratio"],
                "ups": post["ups"],               
                'total_awards_received': post['total_awards_received'],
                "url": post["url"],
                "name": post['name'],
                "author": post['author'],
                "duration": post['duration']
            },            
            ignore_index=True
        )

In [118]:
df_top = df_top.sort_values(["ups", "total_awards_received", "upvote_ratio"], ascending=False, axis=0)

In [119]:
df_top

,title,upvote_ratio,ups,total_awards_received,url,name,author,duration
0,"""What would you like your obituary to say?"" RI...",0.97,5632.0,16.0,https://v.redd.it/i6ghfjfd35i71,t3_p6u5f4,hey_dont_ban_me_bro,26.0
3,Are any women here today?,0.97,2563.0,9.0,https://v.redd.it/3uoltpdsaai71,t3_p7bjxc,NasseHeger,139.0
1,Parkour!,0.88,1277.0,6.0,https://v.redd.it/sy8ola7nb6i71,t3_p6z5nz,tacolamp,5.0
2,"Take a seat, right over here",0.97,951.0,0.0,https://v.redd.it/b4j4gz8wj7i71,t3_p73sa2,trytheCOLDchai,9.0
4,"You're not stuck, I swear.",0.93,546.0,3.0,https://v.redd.it/4ih9eu8ag6i71,t3_p6zogy,Momaroria,9.0
5,7.2 MagniTurd Earthquake,0.84,263.0,0.0,https://v.redd.it/o9bvefm7l7i71,t3_p73x8b,drewd0g,7.0
6,Hilarious and cute at the same moment 😂,0.86,236.0,1.0,https://v.redd.it/gxugpywk36i71,t3_p6y8uq,_Mr_JokeR_,4.0
7,It's always Germany,0.82,98.0,0.0,https://v.redd.it/gpz9e9um55i71,t3_p6uesz,Subtle_srikhand,9.0
8,This is how you almost lose a brand new drone…,0.91,94.0,0.0,https://v.redd.it/08nvzofz59i71,t3_p78upn,LobbethThou,5.0
9,Broke in a bank to make a hot pocket,0.86,43.0,0.0,https://v.redd.it/rq85r5g899i71,t3_p79357,Supermandtm,16.0


In [150]:
df_filtered = pd.DataFrame()

In [153]:
total_duration = 0
for index, row in df_top.iterrows():
    duration = row['duration']
    has_awards = True if row['total_awards_received'] > 0 else False
    
    if total_duration < 600 or  has_awards:    
        total_duration += row['duration']
        df_filtered = df_filtered.append(row)

In [164]:
df_filtered.to_dict('records')

[{'title': '"What would you like your obituary to say?" RIP Sean Lock',
  'upvote_ratio': 0.97,
  'ups': 5632.0,
  'total_awards_received': 16.0,
  'url': 'https://v.redd.it/i6ghfjfd35i71',
  'name': 't3_p6u5f4',
  'author': 'hey_dont_ban_me_bro',
  'duration': 26.0},
 {'title': 'Are any women here today?',
  'upvote_ratio': 0.97,
  'ups': 2563.0,
  'total_awards_received': 9.0,
  'url': 'https://v.redd.it/3uoltpdsaai71',
  'name': 't3_p7bjxc',
  'author': 'NasseHeger',
  'duration': 139.0},
 {'title': 'Parkour!',
  'upvote_ratio': 0.88,
  'ups': 1277.0,
  'total_awards_received': 6.0,
  'url': 'https://v.redd.it/sy8ola7nb6i71',
  'name': 't3_p6z5nz',
  'author': 'tacolamp',
  'duration': 5.0},
 {'title': 'Take a seat, right over here',
  'upvote_ratio': 0.97,
  'ups': 951.0,
  'total_awards_received': 0.0,
  'url': 'https://v.redd.it/b4j4gz8wj7i71',
  'name': 't3_p73sa2',
  'author': 'trytheCOLDchai',
  'duration': 9.0},
 {'title': "You're not stuck, I swear.",
  'upvote_ratio': 0.93,

In [2]:
from helpers import ddb as ddb_helpers
from entities.GatherPosts import GatherPosts
import logging, boto3

ddb = boto3.client('dynamodb')

DAILY_UPLOADS_TABLE_NAME="DailyUploadsTable-dev"
logger = logging.getLogger()

gather_posts = GatherPosts(subreddit='funny', logger=logger)

In [11]:
params = {
        "TransactItems": [
            {
                "Put": {
                    "TableName": DAILY_UPLOADS_TABLE_NAME,
                    "Item": gather_posts.serialize_to_item(),
                }
            },
            {
                "Update": {
                    "TableName": DAILY_UPLOADS_TABLE_NAME,
                    "Key": {
                        "PK": {"S": gather_posts.date},
                        "SK": {"S": "todays_subreddits_count"},
                    },
                    "ConditionExpression": "attribute_exists(PK) and attribute_exists(SK)",
                    "UpdateExpression": "SET #count = #count + :inc, #last_processed_subreddit = :last_processed_subreddit",
                    "ExpressionAttributeNames": {"#count": "count",
                                                 "#last_processed_subreddit": "last_processed_subreddit"},
                    "ExpressionAttributeValues": {":inc": {"N": "1"},
                                                   ":last_processed_subreddit" : {"S": gather_posts.subreddit}},
                }
            },
        ]
    }

In [12]:
ddb_helpers.transact_write_items(ddb, logger, **params)

{'ResponseMetadata': {'RequestId': '4O4RUID46LSIU8SQE0T3BL0ASVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 20 Aug 2021 09:17:45 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4O4RUID46LSIU8SQE0T3BL0ASVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [3]:
key = gather_posts.key()

total_subreddits_key = gather_posts.key()
todays_subreddits_key = gather_posts.key()

total_subreddits_key["SK"]["S"] = "total_subreddits_count"
todays_subreddits_key["SK"]["S"] = "todays_subreddits_count"

params = {
    "TransactItems": [
        {
            "Get": {
                "Key": total_subreddits_key,
                "TableName": DAILY_UPLOADS_TABLE_NAME,
            },
        },
        {
            "Get": {
                "Key": todays_subreddits_key,
                "TableName": DAILY_UPLOADS_TABLE_NAME,
            },
        },
    ]
}

In [4]:
items = ddb_helpers.transact_get_items(ddb, logger, **params)

In [8]:
items[1]

{'SK': {'S': 'todays_subreddits_count'},
 'count': {'N': '1'},
 'PK': {'S': '2021-08-20'},
 'last_processed_subreddit': {'S': 'funny'}}

In [9]:
total_subreddits_item_deserialized = GatherPosts.deserialize_PK_SK_count(items[0])

In [10]:
todays_subreddit_item_deserialized = GatherPosts.deserialize_PK_SK_count(items[1])

In [14]:
total_subreddits_item_deserialized['count'] == todays_subreddit_item_deserialized['count'] and todays_subreddit_item_deserialized['last_processed_subreddit'] == gather_posts.subreddit

True

In [18]:
total_subreddits_item_deserialized, todays_subreddits_item_deserialized = [
        GatherPosts.deserialize_PK_SK_count(item) for item in items
    ]

In [19]:
total_subreddits_item_deserialized

{'SK': 'total_subreddits_count', 'count': '1', 'PK': '2021-08-20'}

In [20]:
todays_subreddit_item_deserialized

{'SK': 'todays_subreddits_count',
 'count': '1',
 'PK': '2021-08-20',
 'last_processed_subreddit': 'funny'}

In [22]:
print (
        total_subreddits_item_deserialized["count"]
        == todays_subreddit_item_deserialized["count"]
        and todays_subreddit_item_deserialized["last_processed_subreddit"]
        == gather_posts.subreddit
    )

True


In [44]:
from helpers import s3 as s3_helpers
import boto3, logging

# Initialize logger and its config.
logger = logging.getLogger()
logger.setLevel(logging.INFO)

s3 = boto3.client('s3')


params = {
    "Bucket":'whodoesntlovereddit-2021-08-21-funny'
}

In [11]:
try:
    resp = s3.head_bucket(**params)
except s3.exceptions.ClientError as e:
    print(dir(e), e.response['Error']['Code'])

['MSG_TEMPLATE', '__cause__', '__class__', '__context__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__suppress_context__', '__traceback__', '__weakref__', '_get_retry_info', 'args', 'operation_name', 'response', 'with_traceback'] 404


In [13]:
def parse_subreddit_group(subreddit_group):
    subreddit_group_arr = subreddit_group.split("-")
    return subreddit_group_arr[4:]



parse_subreddit_group('whodoesntlovereddit-2021-08-21-funny')

['funny']

In [40]:
post = {}
post['url'] = 'https://v.redd.it/t0xqaksf6mi71'
post['name'] = 't3_p8id2y'

In [45]:
bucket_name = 'whodoesntlovereddit-2021-08-21-funny'
file_name = post["name"] + ".mp4"
file_path = "./" + file_name

In [42]:
import youtube_dl
# ydl_opts = {
#             "format": "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best",
#             "logger": logging.getLogger(),
#             "outtmpl": file_path,
#         }

ydl_opts = {
            "format": "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best",
            "logger": logging.getLogger(),
            "outtmpl": file_path,
        }

In [43]:
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([post["url"]])

In [46]:
with open(file_path, "rb") as f:
    s3.upload_fileobj(f, bucket_name, file_name)